In [1]:
!gdown 140gBusnfYFOAaXn5TwGa2rIle_ocsjOP
!gdown 1uaYW_6dcLDAoVpxg2jqM1FVEkNwTKvhB

Downloading...
From (original): https://drive.google.com/uc?id=140gBusnfYFOAaXn5TwGa2rIle_ocsjOP
From (redirected): https://drive.google.com/uc?id=140gBusnfYFOAaXn5TwGa2rIle_ocsjOP&confirm=t&uuid=7f5220bf-a2b2-4748-a480-770a4e9f753f
To: /content/refined2_short.mp4
100% 125M/125M [00:11<00:00, 11.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1uaYW_6dcLDAoVpxg2jqM1FVEkNwTKvhB
To: /content/annotations_ball.label
100% 21.2k/21.2k [00:00<00:00, 47.2MB/s]


In [2]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.1/871.1 kB 17.2 MB/s eta 0:00:00


In [3]:
import cv2
from ultralytics import YOLO
from PIL import Image
import numpy as np

In [39]:
video = 'refined2_short.mp4'
annotation_path = 'annotations_ball.label'

max_yolo_imgsz = None
# yolo = YOLO("weights/best.pt")
yolo = YOLO('yolov8m').cuda()

decrease_size_factor = 0.8
increase_size_factor = 1.2

min_crop_size = 160

In [40]:
counter = 0
annotation = np.loadtxt(annotation_path, delimiter=',', dtype=int)
def get_next_annotation():
    global counter, annotation
    print(f'Progress: {counter+1}/{int(cap.get(cv2.CAP_PROP_FRAME_COUNT))}')
    _, ann_x, ann_y, ann_r = annotation[counter] # center position and radius
    ann_x1 = ann_x - ann_r
    ann_y1 = ann_y - ann_r
    ann_x2 = ann_x + ann_r
    ann_y2 = ann_y + ann_r
    counter += 1
    return ann_x1, ann_y1, ann_x2, ann_y2



def get_yolo_predictions(frame, size):
    image = Image.fromarray(frame)
    results = yolo.predict(image, conf=0.01, classes=[32], imgsz=size)[0]
    conf = results.boxes.conf.tolist()
    boxes = [[int(x1), int(y1), int(x2), int(y2)] for x1, y1, x2, y2 in results.boxes.xyxy.tolist()]
    return boxes, conf


def get_imgsz(x, y):
    global max_yolo_imgsz
    size_x = x - x % 32
    size_y = y - y % 32
    if max_yolo_imgsz is None:
        return int(max(size_x, size_y))
    return min(int(max(size_x, size_y)), max_yolo_imgsz)


def evaluate(box):
    pred_x1, pred_y1, pred_x2, pred_y2 = box
    ann_x1, ann_y1, ann_x2, ann_y2 = get_next_annotation()

    if ann_x1 == 0 and ann_y1 == 0 and ann_x2 == 0 and ann_y2 == 0:
        # no annotation
        if pred_x1 == 0 and pred_y1 == 0 and pred_x2 == 0 and pred_y2 == 0:
            # true negative
            return 0, 0, 0
        else:
            # false positive
            return 0, 1, 0
    else:
        # there is annotation
        if pred_x1 == 0 and pred_y1 == 0 and pred_x2 == 0 and pred_y2 == 0:
            # false negative
            return 0, 0, 1
        else:
            # there is prediction
            if (pred_x1 > ann_x1 and pred_x1 < ann_x2 and pred_y1 > ann_y1 and pred_y1 < ann_y2) or \
                (pred_x2 > ann_x1 and pred_x2 < ann_x2 and pred_y2 > ann_y1 and pred_y2 < ann_y2) or \
                (ann_x1 > pred_x1 and ann_x1 < pred_x2 and ann_y1 > pred_y1 and ann_y1 < pred_y2) or \
                (ann_x2 > pred_x1 and ann_x2 < pred_x2 and ann_y2 > pred_y1 and ann_y2 < pred_y2):
                # true positive
                return 1, 0, 0
            else:
                # false positive
                return 0, 1, 0



cap = cv2.VideoCapture(video)
search_x1 = 0
search_y1 = 0
search_x2 = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
search_y2 = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
tp = 0
fp = 0
fn = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    crop = frame[search_y1:search_y2, search_x1:search_x2]

    imgsz = get_imgsz(crop.shape[1], crop.shape[0])
    boxes, conf = get_yolo_predictions(crop, imgsz)

    if len(boxes) > 0:
        pred_x1, pred_y1, pred_x2, pred_y2 = boxes[0]
        search_x1 = min(max(int(search_x1 * (2 - decrease_size_factor)), 0), frame.shape[1])
        search_y1 = min(max(int(search_y1 * (2 - decrease_size_factor)), 0), frame.shape[0])
        search_x2 = max(min(int(search_x2 * (decrease_size_factor)), frame.shape[1]), 0)
        search_y2 = max(min(int(search_y2 * (decrease_size_factor)), frame.shape[0]), 0)
        pred_center = ((pred_x1 + pred_x2) // 2, (pred_y1 + pred_y2) // 2)
        current_center = ((search_x1 + search_x2) // 2, (search_y1 + search_y2) // 2)
        adjustment = (pred_center[0] - current_center[0], pred_center[1] - current_center[1])
        search_x1 += adjustment[0]
        search_x2 += adjustment[0]
        search_y1 += adjustment[1]
        search_y2 += adjustment[1]
        search_x1 = min(max(search_x1, 0), frame.shape[1])
        search_x2 = min(max(search_x2, 0), frame.shape[1])
        search_y1 = min(max(search_y1, 0), frame.shape[0])
        search_y2 = min(max(search_y2, 0), frame.shape[0])
        if search_x2 - search_x1 < min_crop_size or search_y2 - search_y1 < min_crop_size:
            search_x1 = max(search_x1 - min_crop_size // 2, 0)
            search_y1 = max(search_y1 - min_crop_size // 2, 0)
            search_x2 = min(search_x2 + min_crop_size // 2, frame.shape[1])
            search_y2 = min(search_y2 + min_crop_size // 2, frame.shape[0])
    else:
        pred_x1, pred_y1, pred_x2, pred_y2 = 0, 0, 0, 0
        search_x1 = min(max(int(search_x1 * (2 - increase_size_factor)), 0), frame.shape[1])
        search_y1 = min(max(int(search_y1 * (2 - increase_size_factor)), 0), frame.shape[0])
        search_x2 = max(min(int(search_x2 * (increase_size_factor)), frame.shape[1]), 0)
        search_y2 = max(min(int(search_y2 * (increase_size_factor)), frame.shape[0]), 0)

    is_tp, is_fp, is_fn = evaluate((pred_x1, pred_y1, pred_x2, pred_y2))
    tp += is_tp
    fp += is_fp
    fn += is_fn

cap.release()

if tp+fp == 0:
    precision = 0
else:
    precision = tp / (tp + fp)
if tp+fn == 0:
    recall = 0
else:
    recall = tp / (tp + fn)
if precision+recall == 0:
    f1 = 0
else:
    f1 = 2 * precision * recall / (precision + recall)
print(f'TP: {tp}, FP: {fp}, FN: {fn}')
print(f'Precision: {precision}, Recall: {recall}, F1: {f1}')

Output streaming troncato alle ultime 5000 righe.
Speed: 9.7ms preprocess, 91.2ms inference, 1.1ms postprocess per image at shape (1, 3, 2560, 704)
Progress: 562/1811

0: 2816x832 (no detections), 121.4ms
Speed: 13.2ms preprocess, 121.4ms inference, 1.0ms postprocess per image at shape (1, 3, 2816, 832)
Progress: 563/1811

0: 3040x928 1 sports ball, 147.0ms
Speed: 14.6ms preprocess, 147.0ms inference, 1.8ms postprocess per image at shape (1, 3, 3040, 928)
Progress: 564/1811

0: 2080x576 (no detections), 54.4ms
Speed: 6.7ms preprocess, 54.4ms inference, 1.1ms postprocess per image at shape (1, 3, 2080, 576)
Progress: 565/1811

0: 2848x864 1 sports ball, 130.2ms
Speed: 12.8ms preprocess, 130.2ms inference, 1.5ms postprocess per image at shape (1, 3, 2848, 864)
Progress: 566/1811

0: 1984x544 (no detections), 54.0ms
Speed: 5.6ms preprocess, 54.0ms inference, 1.3ms postprocess per image at shape (1, 3, 1984, 544)
Progress: 567/1811

0: 2816x864 (no detections), 124.5ms
Speed: 12.9ms prepro